In [12]:
 !pip install langchain langchain-groq chromadb langchain_community beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [4]:
from langchain_groq import ChatGroq
from google.colab import userdata
import os

os.environ["GROQ_API_KEY"] = userdata.get("groq_api")

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key=os.environ["GROQ_API_KEY"],
    verbose=True,
)

In [10]:
response=llm.invoke("The first person to land on the moon was..")
response.content

'The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon\'s surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That\'s one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.'

In [15]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://jobs.nike.com/job/R-48606")
page_data=loader.load().pop().page_content
page_data

"Apply for Retail Store Manager - Winter Garden\n\nSearch JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsRetail StoresRetail Store Manager - Winter GardenOrlando, FloridaBecome a Part of the N

In [17]:
from langchain.prompts import  PromptTemplate
prompt_extract= PromptTemplate.from_template(
    """
    ### Scraped TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job posting and return them in JSON format containing
    following keys: `role`,`experience`,`skills` and `description`,
    only return the JSON.
    ### VALID JSON(NO PREAMBLE)
    """
)

chain_extract=prompt_extract | llm
res=chain_extract.invoke(input={'page_data':page_data})
print(res.content)
print(type(res.content))

```json
{
    "role": "Retail Store Manager",
    "experience": "Minimum of six years of customer-facing retail or hospitality experience, Minimum of four years of store management experience",
    "skills": [
        "Retail operations",
        "Budgeting",
        "Planning",
        "Customer service",
        "Sales",
        "People leadership",
        "Microsoft Office",
        "Retail business systems",
        "Conflict resolution",
        "Employee coaching and counseling"
    ],
    "description": "As a Nike Store Manager, you’re the face of NIKE. Enjoy high-volume and a fast pace as your diverse experience and perspective helps guide customers in making the best decisions for them. You’ll work with your team to focus on customer service and get to the win the right way."
}
```
<class 'str'>


In [20]:
from langchain_core.output_parsers import JsonOutputParser

json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res

{'role': 'Retail Store Manager',
 'experience': 'Minimum of six years of customer-facing retail or hospitality experience, Minimum of four years of store management experience',
 'skills': ['Retail operations',
  'Budgeting',
  'Planning',
  'Customer service',
  'Sales',
  'People leadership',
  'Microsoft Office',
  'Retail business systems',
  'Conflict resolution',
  'Employee coaching and counseling'],
 'description': 'As a Nike Store Manager, you’re the face of NIKE. Enjoy high-volume and a fast pace as your diverse experience and perspective helps guide customers in making the best decisions for them. You’ll work with your team to focus on customer service and get to the win the right way.'}

In [21]:
type(json_res)

dict

In [23]:
import pandas as pd
df=pd.read_csv('/content/my_portfolio.csv')
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [25]:
import chromadb
import uuid

client=chromadb.PersistentClient('vectorstore')
collection=client.create_collection(name="portfolio")


if not collection.count():
  for _, row in df.iterrows():
    collection.add(documents=row["Techstack"],
                   metadatas={"links":row["Links"]},
                   ids=[str(uuid.uuid4())])

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 84.2MiB/s]


In [28]:
links = collection.query(query_texts=["Experience in python","Expertise in React"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [29]:
job=json_res
job['skills']

['Retail operations',
 'Budgeting',
 'Planning',
 'Customer service',
 'Sales',
 'People leadership',
 'Microsoft Office',
 'Retail business systems',
 'Conflict resolution',
 'Employee coaching and counseling']

In [30]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}]

In [32]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Sam, a Data science and AI student at M.G.R Educational and Research institute. Done internship in data science field and doing AI engineer intern at infosys springboard
        Over my experience, I have completed hands on projects in data science and Machine learning and GEN AI fields
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Sam, Student interested In data science and AI
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Revolutionizing Retail Operations with AtliQ's Expertise

Dear Hiring Manager,

I came across the Retail Store Manager position at Nike and was impressed by the company's commitment to delivering exceptional customer experiences. As a Data Science and AI student with a passion for driving business growth through innovative solutions, I'd like to introduce you to AtliQ, a cutting-edge technology firm that can help elevate your retail operations.

With a strong foundation in data science and machine learning, AtliQ can help Nike optimize its retail strategies, enhance customer engagement, and streamline store management. Our expertise in retail operations, budgeting, and planning can be leveraged to improve sales, customer service, and people leadership. We can also provide tailored solutions for conflict resolution and employee coaching, ensuring a positive and productive work environment.

AtliQ's portfolio showcases our capabilities in developing and implementing technology s